# Timer

In [1]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
# pattern = ['STOCH', 'RSI']
pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 300
load = True

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [20, 25, 30, 35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [15, 20, 25]},
                'LinearReg': {'timeperiod': [4, 6, 8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 15m/1h, trade type is buy
Number of combinations is 60


  0%|                                                                                                                                                                                                   | 0/60 [00:00<?, ?it/s]


Load the datasets...
Binance
ByBit
BinanceFutures


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [05:52<00:00,  5.87s/it]


# Check local statistics

In [3]:
# 15m/1h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
55,STOCH_RSI_LinearReg,14,35,9,7,3,25,4,0,2.8591,5.3510,5.5301,5.7007,5.4982,5.3345,5.4575,5.4258,5.4233,5.2048,5.8701,6.4162,6.3833,6.4108,6.4339,6.4339,6.4339,6.5279,6.6275,6.6093,6.6252,6.7418,6.8428,7.0306,0.34,0.49,0.69,0.67,0.78,0.87,0.67,0.66,0.67,0.70,0.90,1.05,0.98,0.93,0.92,0.85,0.83,0.79,0.88,1.04,0.89,0.89,1.00,0.95,18,5.965508,0.810000,89.379150,14.580000
50,STOCH_RSI_LinearReg,14,35,9,7,3,20,4,0,2.7539,4.8912,4.8519,5.1092,5.0279,4.9100,5.0331,5.0123,5.0166,4.8232,5.5156,6.0850,6.0227,6.0520,6.0752,6.0752,6.0752,6.1691,6.2688,6.2530,6.2758,6.3925,6.4934,6.6495,0.34,0.43,0.65,0.65,0.77,0.83,0.67,0.65,0.62,0.53,0.86,1.04,0.95,0.87,0.91,0.85,0.81,0.77,0.85,0.91,0.86,0.87,0.93,0.92,17,5.576346,0.772500,77.797879,13.132500
45,STOCH_RSI_LinearReg,14,35,9,7,3,15,4,0,4.1303,6.1077,5.2213,5.2657,6.4666,6.3414,6.3836,6.2823,6.1777,5.7103,6.7302,7.4964,7.3770,7.3584,7.3943,7.3943,7.3943,7.3943,7.5384,7.5010,7.3635,7.4970,7.5415,7.6502,0.52,0.43,0.65,0.65,0.79,0.70,0.67,0.65,0.62,0.53,0.86,1.04,0.95,0.87,0.91,0.85,0.81,0.77,0.85,0.91,0.86,0.87,0.93,0.92,13,6.738237,0.775417,74.597087,10.080417
59,STOCH_RSI_LinearReg,14,35,9,7,3,25,12,0,2.4639,2.6731,2.8671,2.9039,3.3494,3.2975,3.2984,3.2878,3.5273,3.3568,3.8178,4.0210,3.8805,3.9160,3.9269,3.8569,3.7978,3.8122,3.8252,3.7808,3.8226,3.8063,3.8183,3.8152,0.45,0.35,0.47,0.53,0.60,0.53,0.56,0.65,0.62,0.57,0.71,0.73,0.83,0.71,0.80,0.85,0.85,0.79,0.71,0.83,0.81,0.76,0.76,0.95,29,3.538446,0.684167,73.614929,19.840833
54,STOCH_RSI_LinearReg,14,35,9,7,3,20,12,0,2.7476,2.7898,2.7687,2.8197,3.3476,3.2878,3.2892,3.2773,3.5505,3.4120,3.9632,4.1762,4.0214,4.0603,4.0726,3.9908,3.9990,4.0306,4.0462,3.9893,4.0373,4.0167,4.0311,4.0074,0.46,0.36,0.51,0.55,0.58,0.54,0.60,0.67,0.67,0.55,0.72,0.76,0.84,0.72,0.83,0.85,0.78,0.69,0.68,0.84,0.82,0.79,0.85,0.95,26,3.655513,0.692083,69.043325,17.994167
56,STOCH_RSI_LinearReg,14,35,9,7,3,25,6,0,2.4726,4.1071,4.0050,4.2891,4.3736,4.3693,4.2377,4.1542,4.1472,3.7392,4.3218,4.6760,4.5899,4.6166,4.6333,4.5964,4.5964,4.5964,4.5964,4.5884,4.5647,4.5325,4.5325,4.5233,0.33,0.43,0.58,0.56,0.59,0.66,0.63,0.61,0.59,0.48,0.63,0.95,0.91,0.86,0.84,0.84,0.74,0.75,0.77,0.80,0.82,0.80,0.93,0.89,20,4.327483,0.707917,66.549667,14.158333
57,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,2.2570,3.6726,3.5040,3.7965,3.9352,3.7005,3.6657,3.6220,4.0263,3.6492,4.1235,4.2279,4.0756,4.0764,4.0899,4.0003,4.0092,4.0437,4.0526,4.0483,4.0755,4.0526,4.0526,4.0480,0.40,0.40,0.46,0.49,0.55,0.54,0.60,0.61,0.58,0.45,0.43,0.82,0.84,0.79,0.86,0.85,0.70,0.69,0.65,0.86,0.83,0.74,0.86,0.94,22,3.866879,0.664167,63.071342,14.611667
46,STOCH_RSI_LinearReg,14,35,9,7,3,15,6,0,3.2885,4.3993,3.4845,3.7219,4.7401,4.8190,4.5689,4.4207,4.4043,4.1747,5.1246,5.9248,5.8531,5.8484,5.8757,5.8757,5.8757,5.8757,5.8757,5.8525,5.7848,5.7185,5.7185,5.6538,0.52,0.43,0.60,0.56,0.62,0.62,0.65,0.65,0.62,0.53,0.86,1.04,0.95,0.87,0.91,0.85,0.75,0.77,0.83,0.86,0.85,0.87,0.93,0.92,15,5.119975,0.752500,61.799625,11.287500
58,STOCH_RSI_LinearReg,14,35,9,7,3,25,10,0,2.5434,3.2640,3.1897,3.1029,3.2947,3.1751,3.1650,3.1396,3.3870,3.1230,3.4973,3.6435,3.51

In [6]:
# 15m/4h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,forecast_rank,e_ratio_rank,price_rank
59,STOCH_RSI_LinearReg,14,35,9,7,3,25,12,0,1.7410,2.3498,2.2167,2.3211,2.5890,2.6798,2.8074,2.8153,2.8713,2.7573,3.0144,3.2206,3.1942,3.1778,3.2808,3.1586,3.1917,3.2839,3.2539,3.0365,3.0947,3.0661,3.0733,3.1077,0.18,0.31,0.28,0.32,0.32,0.36,0.42,0.45,0.42,0.47,0.40,0.50,0.63,0.58,0.61,0.71,0.65,0.65,0.65,0.70,0.71,0.72,0.75,0.82,67,2.887621,0.525417,126.470596,126.470596,35.202917
58,STOCH_RSI_LinearReg,14,35,9,7,3,25,10,0,1.7259,2.2359,2.0820,2.2009,2.4828,2.5857,2.6692,2.6601,2.7188,2.6063,2.8491,3.0639,3.0430,3.0299,3.1358,3.0186,3.0532,3.1464,3.1012,2.8935,2.9479,2.9558,2.9464,2.9562,0.20,0.29,0.25,0.22,0.28,0.34,0.42,0.44,0.40,0.45,0.37,0.44,0.55,0.50,0.55,0.65,0.57,0.62,0.65,0.69,0.67,0.70,0.73,0.81,64,2.754521,0.491250,112.289333,112.289333,31.440000
56,STOCH_RSI_LinearReg,14,35,9,7,3,25,6,0,1.9597,2.5047,2.5413,2.6814,2.9700,3.0492,3.1446,3.1078,3.0902,2.8834,3.1225,3.4012,3.3066,3.3035,3.4439,3.2805,3.3108,3.3874,3.3138,3.0264,3.0809,3.0883,3.0768,3.0873,0.23,0.32,0.28,0.33,0.32,0.42,0.44,0.47,0.44,0.46,0.35,0.44,0.49,0.45,0.50,0.58,0.52,0.59,0.65,0.67,0.61,0.67,0.68,0.75,54,3.048425,0.485833,110.614950,110.614950,26.235000
57,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,1.8510,2.4131,2.2673,2.4085,2.6900,2.7808,2.8681,2.8333,2.8385,2.6730,2.9385,3.1302,3.0854,3.0802,3.2077,3.0766,3.1138,3.1997,3.1492,2.9163,2.9737,2.9815,2.9713,2.9816,0.20,0.32,0.27,0.29,0.31,0.41,0.42,0.45,0.40,0.42,0.35,0.38,0.41,0.38,0.48,0.51,0.49,0.58,0.65,0.69,0.67,0.70,0.70,0.79,58,2.851221,0.469583,107.370808,107.370808,27.235833
54,STOCH_RSI_LinearReg,14,35,9,7,3,20,12,0,1.5385,2.2863,2.0030,2.1851,2.5149,2.6219,2.7430,2.7582,2.8032,2.7705,3.1088,3.3529,3.3287,3.3283,3.4368,3.2829,3.3162,3.3782,3.4165,3.1167,3.1804,3.1662,3.1877,3.2236,0.18,0.31,0.24,0.25,0.26,0.36,0.42,0.46,0.55,0.50,0.56,0.66,0.72,0.61,0.69,0.81,0.72,0.65,0.66,0.70,0.71,0.73,0.76,0.86,53,2.918688,0.557083,101.690438,101.690438,29.525417
51,STOCH_RSI_LinearReg,14,35,9,7,3,20,6,0,1.8120,2.5308,2.3698,2.6606,3.0556,3.1308,3.2042,3.1649,3.1098,2.9840,3.3208,3.6688,3.5570,3.5800,3.7498,3.5251,3.5562,3.5953,3.5959,3.1678,3.2280,3.2681,3.2687,3.2723,0.21,0.33,0.27,0.32,0.31,0.46,0.46,0.56,0.59,0.50,0.40,0.61,0.63,0.56,0.61,0.75,0.65,0.59,0.65,0.67,0.63,0.68,0.69,0.82,41,3.182346,0.539583,89.476179,89.476179,22.122917
53,STOCH_RSI_LinearReg,14,35,9,7,3,20,10,0,1.5123,2.1364,1.8344,2.0347,2.3843,2.5085,2.5765,2.5709,2.6204,2.5873,2.9084,3.1648,3.1459,3.1482,3.2607,3.1140,3.1480,3.2058,3.2224,2.9368,2.9961,3.0299,3.0303,3.0351,0.18,0.29,0.19,0.16,0.22,0.34,0.42,0.45,0.53,0.48,0.44,0.58,0.64,0.57,0.62,0.73,0.66,0.62,0.66,0.69,0.67,0.70,0.76,0.86,50,2.754671,0.519167,87.733542,87.733542,25.958333
52,STOCH_RSI_LinearReg,14,35,9,7,3,20,8,0,1.6853,2.4038,2.0552,2.3159,2.6913,2.7909,2.8580,2.8208,2.8018,2.7122,3.0700,3.2987,3.2493,3.2635,3.4140,3.2422,3.2797,3.3270,3.3442,3.0104,3.0738,3.1100,3.1105,3.1151,0.18,0.33,0.24,0.25,0.26,0.42,0.42,0.47,0.55,0.47,0.40,0.56,0.56,0.44,0.58,0.69,0.59,0.56,0.65,0.70,0.71,0.73,0.75,0.86,45,2.918483,0.515417,86.331750,86.331750,23.193750
55,STOCH_RSI_LinearReg,14

# Save new config data to config file

In [6]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [6], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            